# Day 8: Seven Segment Search

https://adventofcode.com/2021/day/8

## Part 1

In [1]:
short_example_txt = "acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | cdfeb fcadb cdfeb cdbaf"

In [2]:
long_example_txt = """be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce"""

In [3]:
with open('input.txt') as input_file:
    input_txt = input_file.read()

In [4]:
def process_input(txt):
    """Manipulate the input text into two lists."""
    txt = txt.strip().split('\n')
    signal_patterns, output_values = [], []
    for entry in txt:
        signal, output = entry.strip().split('|')
        signal_patterns.append(signal.strip().split())
        output_values.append(output.strip().split())
    return signal_patterns, output_values

Define the seven segments for each digit from 0 to 9.  Use capital letters for the original segments before the malfunction.

In [5]:
correct_patterns = ['ABCEFG', 'CF', 'ACDEG', 'ACDFG', 'BCDF', 'ABDFG', 'ABDEFG', 'ACF', 'ABCDEFG', 'ABCDFG']
for i, pattern in enumerate(correct_patterns):
    print(i, pattern)

0 ABCEFG
1 CF
2 ACDEG
3 ACDFG
4 BCDF
5 ABDFG
6 ABDEFG
7 ACF
8 ABCDEFG
9 ABCDFG


In [6]:
def count_easy_digits(output_values, correct_patterns):
    """Number of easy digits (1, 4, 7, or 8) in output values."""
    easy_digits = (1, 4, 7, 8)
    count = 0
    for entry in range(len(output_values)):
        for value in output_values[entry]:
            for easy_digit in (1, 4, 7, 8):
                if len(correct_patterns[easy_digit]) == len(value):
                    count += 1
    print(f'There are {count} instances of digits {easy_digits}.')

In [7]:
for txt in (short_example_txt, long_example_txt, input_txt):
    signal_patterns, output_values = process_input(txt)
    count_easy_digits(output_values, correct_patterns)

There are 0 instances of digits (1, 4, 7, 8).
There are 26 instances of digits (1, 4, 7, 8).
There are 530 instances of digits (1, 4, 7, 8).


## Part 2

Use the length of the patterns to disentangle the signal.

In [8]:
for digit, pattern in enumerate(correct_patterns):
    print(digit, pattern, len(pattern))

0 ABCEFG 6
1 CF 2
2 ACDEG 5
3 ACDFG 5
4 BCDF 4
5 ABDFG 5
6 ABDEFG 6
7 ACF 3
8 ABCDEFG 7
9 ABCDFG 6


In [9]:
def get_incorrect_patterns(patterns):
    """Returns the seven segments for each digit after the malfunction."""
    
    incorrect_patterns = 10*['']  # placeholder until the patterns are filled
    
    # First the easy digits that are uniquely identified by the pattern length.
    for pattern in patterns:
        for digit in (1, 4, 7, 8):
            if len(pattern) == len(correct_patterns[digit]):
                incorrect_patterns[digit] = pattern        
                
    # The remaining 6 digits can be divided into 3 with length 5 and 3 with length 6.
    # Each set of 3 digits can be assigned by identifying substrings within the pattern.

    # There are three patterns of length 6: 0 = ABCEFG, 6 = ABDEFG, 9 = ABCDFG.
    for pattern in [p for p in patterns if len(p) == 6]:

        if all([char in pattern for char in incorrect_patterns[4]]):
            # 4 = BCDF only appears in 9 = ABCDFG, not in 0 = ABCEFG or 6 = ABDEFG
            digit = 9

        elif all([char in pattern for char in incorrect_patterns[1]]):  # CF
            # 1 = CF only appears in 0 = ABCEFG, not in 6 = ABDEFG
            digit = 0

        else:  # 6 = ABDEFG
            digit = 6

        incorrect_patterns[digit] = pattern

    # There are three patterns of length 5: 2 = ACDEG, 3 = ACDFG, 5 = ABDFG.
    for pattern in [p for p in patterns if len(p) == 5]:
            
        if all([char in pattern for char in incorrect_patterns[1]]):
            # 1 = CF only appears in 3 = ACDFG, not 2 = ACDEG or 5 = ABDFG
            digit = 3

        elif all([char in incorrect_patterns[6] for char in pattern]):
            # only 5 = ABDFG (not 2 = ACDEG) appears in 6 = ABDEFG
            digit = 5

        else:
            digit = 2  # 2 = ACDEG

        incorrect_patterns[digit] = pattern
            
    # Sort the patterns into alphabetical order.
    for i, pattern in enumerate(incorrect_patterns):
        incorrect_patterns[i] = ''.join(sorted(pattern))

    return incorrect_patterns

Trivial check that the function works given the correct patterns.

In [10]:
assert get_incorrect_patterns(correct_patterns) == correct_patterns

Evaulate the sum of output numbers with the short and long examples, then the input text file.

In [11]:
for txt in (short_example_txt, long_example_txt, input_txt):
    output_number = 0
    signal_patterns, output_values = process_input(txt)
    for i, entry in enumerate(output_values):
        output_string = ''
        incorrect_patterns = get_incorrect_patterns(signal_patterns[i])
        for value in entry:
            output_string += str(incorrect_patterns.index(''.join(sorted(value))))
        output_number += int(output_string)
    print(f'Sum of four-digit output values is {output_number}.')

Sum of four-digit output values is 5353.
Sum of four-digit output values is 61229.
Sum of four-digit output values is 1051087.
